## Tescan Testing Notebook

#### Installation Instructions

1. Download fibsem, autolamella
2. Install VSCode
3. Install Miniconda
4. Create openfibsem env python=3.9/3.11 pip (conda create -n openfibsem python=3.9 pip)
5. Install SharkSEM, SharkSEMAutomation in openfibsem environment
6. Install fibsem, autolamella into the openfibsem environment (pip install -e .[ui])

Ready to Run

### Microscope Connection and Configuration

In [ ]:
%load_ext autoreload
%autoreload 2

import os
from pprint import pprint

import matplotlib.pyplot as plt

from fibsem import acquire, utils
from fibsem.config import CONFIG_PATH
from fibsem.microscopes.tescan import TescanMicroscope
from fibsem.structures import BeamType, FibsemStagePosition

TESCAN_CONFIGURATION = os.path.join(CONFIG_PATH, "tescan-configuration.yaml")

microscope: TescanMicroscope
microscope, settings = utils.setup_session(config_path=TESCAN_CONFIGURATION)

In [ ]:
def plot_image(image, title=None, crosshair=True):
    plt.figure(figsize=(10, 10))
    plt.imshow(image.data, cmap='gray')
    if title:
        plt.title(title)
    if crosshair:
        plt.hlines(image.data.shape[0] // 2, 0, image.data.shape[1], colors='yellow', linestyles='dashed')
        plt.vlines(image.data.shape[1] // 2, 0, image.data.shape[0], colors='yellow', linestyles='dashed')
    plt.axis('off')
    plt.show()


def plot_images(images, titles=None):
    n = len(images)
    fig, axs = plt.subplots(1, n, figsize=(5 * n, 5))
    for i, image in enumerate(images):
        axs[i].imshow(image.data, cmap='gray')
        if titles:
            axs[i].set_title(titles[i])
        axs[i].axis('off')
    
        axs[i].hlines(image.data.shape[0] // 2, 0, image.data.shape[1], colors='yellow', linestyles='dashed')
        axs[i].vlines(image.data.shape[1] // 2, 0, image.data.shape[0], colors='yellow', linestyles='dashed')
    plt.tight_layout()
    plt.show()

In [ ]:
# Get microscope configuration

# manufacturer info
print("Manufacturer Info:")
pprint(microscope.system.info.to_dict())

print("\nStage Configuration:")
pprint(microscope.system.stage.to_dict())

print(f"\nSEM Configuration - Column Tilt: {microscope.system.electron.column_tilt}")
print(f"\nFIB Configuration - Column Tilt: {microscope.system.ion.column_tilt}")

### Image Acquisition

In [ ]:
settings.image.hfw = 200e-6
settings.image.dwell_time = 0.2e-6
settings.image.autocontrast = False
settings.image.beam_type = BeamType.ELECTRON

sem_image = acquire.acquire_image(microscope, settings.image)

plot_image(sem_image, title="SEM Image", crosshair=True)


In [ ]:
settings.image.beam_type = BeamType.ION
fib_image = acquire.acquire_image(microscope, settings.image)

# Display the acquired FIB image
plot_image(fib_image, title="FIB Image", crosshair=True)


In [ ]:
# Acquire both SEM and FIB images
images = acquire.take_reference_images(microscope, settings.image)

plot_images(images, titles=["SEM Image", "FIB Image"])

### Stage Position and Movement

In [ ]:
stage_position = microscope.get_stage_position()
print(f"\nStage Position: {stage_position}")

stage_orientation = microscope.get_stage_orientation()
print(f"\nStage Orientation: {stage_orientation}")

In [ ]:
# relative movement

sem_image1, fib_image1 = acquire.take_reference_images(microscope, settings.image)
microscope.move_stage_relative(FibsemStagePosition(x=20e-6))
sem_image2, fib_image2 = acquire.take_reference_images(microscope, settings.image)

plot_images([sem_image1, fib_image1], titles=["SEM Image 1", "FIB Image 1"])
plot_images([sem_image2, fib_image2], titles=["SEM Image 2", "FIB Image 2"])

In [ ]:
# absolute movement
initial_position = microscope.get_stage_position()

# move relative dx=50e-6, dy=50e-6
microscope.move_stage_relative(FibsemStagePosition(x=50e-6, y=50e-6))
end_position = microscope.get_stage_position()

print(f"\nInitial Stage Position: {initial_position}")
print(f"End Stage Position: {end_position}")
microscope.move_stage_absolute(initial_position)
print(f"Moved back to Initial Stage Position: {microscope.get_stage_position()}")

In [ ]:
# target orientation
current_position = microscope.get_stage_position()
target_position = microscope.get_target_position(current_position, "SEM")
microscope.safe_absolute_stage_movement(target_position)

In [ ]:
# stable movement
initial_position = microscope.get_stage_position()
sem_image1, fib_image1 = acquire.take_reference_images(microscope, settings.image)
microscope.stable_move(dx=50e-6, dy=50e-6, beam_type=BeamType.ELECTRON)

final_position = microscope.get_stage_position()
sem_image2, fib_image2 = acquire.take_reference_images(microscope, settings.image)

print(f"\nInitial Position: {initial_position}")
print(f"Final Position: {final_position}")

plot_images([sem_image1, fib_image1], titles=["SEM Image 1", "FIB Image 1"])
plot_images([sem_image2, fib_image2], titles=["SEM Image 2", "FIB Image 2"])

In [ ]:
# vertical stage movement

initial_position = microscope.get_stage_position()
sem_image1, fib_image1 = acquire.take_reference_images(microscope, settings.image)
microscope.vertical_move(dy=50e-6)
final_position = microscope.get_stage_position()
sem_image2, fib_image2 = acquire.take_reference_images(microscope, settings.image)

print(f"\nInitial Position: {initial_position}")
print(f"Final Position: {final_position}")

plot_images([sem_image1, fib_image1], titles=["SEM Image 1", "FIB Image 1"])
plot_images([sem_image2, fib_image2], titles=["SEM Image 2", "FIB Image 2"])

### Beam Shift Alignment


In [ ]:
import numpy as np

# NOTE: on simulator, beam shift is applied before scan rotation. i.e. it moves in opposite direction when scan rotation is applied.

for sr in [0, 180]:
    microscope.set_scan_rotation(sr, BeamType.ION)

    microscope.reset_beam_shifts()
    initial_beam_shift = microscope.get_beam_shift(BeamType.ION)

    settings.image.beam_type = BeamType.ION
    fib_image1 = acquire.acquire_image(microscope, settings.image)
    microscope.beam_shift(dx=0e-6, dy=10e-6, beam_type=BeamType.ION)
    fib_image2 = acquire.acquire_image(microscope, settings.image)
    final_beam_shift = microscope.get_beam_shift(BeamType.ION)

    plot_images([fib_image1, fib_image2], titles=["FIB Image 1", "FIB Image 2"])
    print(f"Scan Rotation: {sr} degrees")
    print(f"Initial Beam Shift: {initial_beam_shift}")
    print(f"Final Beam Shift: {final_beam_shift}")

    microscope.reset_beam_shifts()

In [ ]:
from fibsem import alignment
from fibsem.structures import FibsemRectangle


for sr in [0, 180]:
    print(f"\nApplying Scan Rotation: {sr} degrees")
    microscope.set_scan_rotation(sr, BeamType.ION)
    # reset beam shifts before alignment
    microscope.reset_beam_shifts()

    # Acquire a reference image with reduced area for beam shift alignment
    settings.image.reduced_area = FibsemRectangle(0.25, 0.25, 0.5, 0.5) # centred bbox
    settings.image.beam_type = BeamType.ION
    ref_image = acquire.acquire_image(microscope, settings.image)

    # randomly apply beam shift to ION
    microscope.beam_shift(dx=10e-6, dy=10e-6, beam_type=BeamType.ION)
    shifted_image = acquire.acquire_image(microscope, settings.image)

    # Perform beam shift alignment
    alignment.multi_step_alignment_v2(microscope, ref_image, beam_type=BeamType.ION, steps=3)

    # Verify the alignment by acquiring another image
    aligned_image = acquire.acquire_image(microscope, settings.image)

    # Plot the images to compare
    plot_images([ref_image, shifted_image, aligned_image], 
                titles=["Reference Image", "Shifted Image", "Aligned Image"])

    print("Beam shift alignment completed successfully.")
    print(f"Final Beam Shift: {microscope.get_beam_shift(BeamType.ION)}")

### Milling

In [ ]:
from fibsem.milling import mill_stages, FibsemMillingStage
from fibsem.milling.base import FibsemMillingSettings
from fibsem.milling.patterning.patterns2 import RectanglePattern
from fibsem.milling.patterning.plotting import draw_milling_patterns


# Define milling settings and pattern
milling_settings = FibsemMillingSettings(
    hfw=150e-6,
    preset="30 keV; 10 nA",
    rate=30.0e-11,  # in um^3/s
)

rect_pattern = RectanglePattern(
    width=20e-6,
    height=10e-6,
    depth=0.5e-6
)
    
milling_stage = FibsemMillingStage(name="Tescan Test", 
                                   milling=milling_settings, pattern=rect_pattern,)
milling_stage.alignment.enabled = False


In [ ]:
# acquire the milling stage image
settings.image.beam_type = BeamType.ION
settings.image.hfw = milling_stage.milling.hfw
fib_image = acquire.acquire_image(microscope, settings.image)

In [ ]:
fig = draw_milling_patterns(image=fib_image, 
                            milling_stages=[milling_stage], 
                            title="Milling Patterns", show_preset=True )
plt.show()

In [ ]:
# run milling 
mill_stages(microscope=microscope, stages=[milling_stage])

In [ ]:
from autolamella.config import PROTOCOL_PATH
from autolamella.structures import AutoLamellaProtocol

protocol = AutoLamellaProtocol.load(PROTOCOL_PATH)

print(protocol.milling)

rough_milling_stages = protocol.milling["mill_rough"]

# acquire the rough milling stage image
settings.image.hfw = rough_milling_stages[0].milling.hfw

fib_image = acquire.acquire_image(microscope, settings.image)
fig = draw_milling_patterns(image=fib_image,
                            milling_stages=rough_milling_stages,
                            title="Rough Milling Stages", show_preset=True)


In [ ]:
mill_stages(microscope=microscope, stages=rough_milling_stages)